In [6]:
# get a random statement to test attn mechanism
text='Peter Piper picked a peck of pickled peppers. A peck of pickled peppers Peter Piper picked'

In [52]:
import torch
from src.data_prep.data_load import CustomDataLoad
import tiktoken
import torch.nn as nn

In [ ]:
#tokenize the input convert them into input output batches
tk=tiktoken.get_encoding('gpt2')
dl=CustomDataLoad(text,4,1,4,False,True,0)

In [15]:
dataLoad=dl.get_data_loader()

In [17]:
cur=iter(dataLoad)
input,target=next(cur)
print(input)

tensor([[19727, 33503,  6497,   257]])


In [23]:
#embed context to each token of 256 for demonstration add a random embeeding 
lookup=torch.nn.Embedding(50257,8)#vocab size of tiktoken for gpt2
print(lookup.weight)


Parameter containing:
tensor([[-0.3684,  1.8227,  0.6509,  ..., -1.3220,  0.4791, -0.5762],
        [ 0.8342,  0.0056,  1.9680,  ..., -0.3135,  0.0548,  0.4535],
        [ 2.2243, -0.1519,  2.0594,  ..., -0.1665,  2.0036, -0.2256],
        ...,
        [-0.3729,  0.4732, -0.6389,  ..., -0.8147, -1.3889, -1.3743],
        [ 0.6490,  0.7061, -0.5395,  ...,  1.5319, -0.2826, -1.4414],
        [-0.1150, -2.9221, -0.5715,  ..., -0.1242, -1.3125,  1.3879]],
       requires_grad=True)


In [32]:
#use one input to demostrate attn mechanism
data_embed=lookup(input)
flattened=data_embed.reshape(4,8)
print(flattened.shape)

torch.Size([4, 8])


In [40]:
#take the target token and perform dot token to identify the similarity between 
# the two tensors by performing dot product then perform normaliztion to keep velues bw 0 and 1 also use softmax to avoid overflow errors
target=flattened[0]
attn_score=torch.empty(flattened.shape[0])
for i in range(len(flattened)):
    attn_score[i]=torch.dot(target,flattened[i])
attn_weights=torch.softmax(attn_score,dim=0)
print(attn_score,attn_weights)
    

tensor([3.7546, 1.2107, 2.6435, 0.3704], grad_fn=<CopySlices>) tensor([0.6936, 0.0545, 0.2283, 0.0235], grad_fn=<SoftmaxBackward0>)


In [41]:
# Now multiply the dot matrix normalized values to the respective tensors and get the sum of all the vectors 
# in the input to obtain context
attn_vec=torch.zeros(target.shape)
for i in range(len(attn_weights)):
    attn_vec+=flattened[i]*attn_weights[i]
print(attn_vec)


tensor([-1.4206,  0.2713,  0.1409,  0.0018, -0.4920, -0.6390, -0.4650, -0.3945],
       grad_fn=<AddBackward0>)


In [42]:
# To get a attn matrix we have to do the same for all tokens
#the multiplication of each token vector with every other vector can be accomplished by performing matrix multiplication
attn_metrics=flattened @ flattened.T
print(attn_metrics)

tensor([[ 3.7546,  1.2107,  2.6435,  0.3704],
        [ 1.2107,  4.7110,  0.3374, -2.1224],
        [ 2.6435,  0.3374,  6.6409,  1.7895],
        [ 0.3704, -2.1224,  1.7895,  6.8002]], grad_fn=<MmBackward0>)


In [43]:
# To keep the values bw 0 and +1 so we can multiply the weights with the input matrix perform normalization
attn_weights=torch.softmax(attn_metrics,dim=-1)
print(attn_weights)

tensor([[6.9365e-01, 5.4489e-02, 2.2835e-01, 2.3517e-02],
        [2.8918e-02, 9.5797e-01, 1.2076e-02, 1.0319e-03],
        [1.7863e-02, 1.7800e-03, 9.7275e-01, 7.6044e-03],
        [1.5994e-03, 1.3223e-04, 6.6110e-03, 9.9166e-01]],
       grad_fn=<SoftmaxBackward0>)


In [45]:
torch.sum(attn_weights[0])

tensor(1., grad_fn=<SumBackward0>)

In [47]:
#now to finally obtain the weights we multipy the scores with the input values
final_weights=attn_weights @ flattened
print(final_weights)

tensor([[-1.4206,  0.2713,  0.1409,  0.0018, -0.4920, -0.6390, -0.4650, -0.3945],
        [-0.0045,  0.7083,  1.3062,  0.7882, -0.1093, -0.3308, -0.2067, -1.1861],
        [-1.2075,  0.7773,  0.1682, -0.1800, -0.7364, -1.6026, -0.7583,  0.7726],
        [-0.7423, -1.4619,  0.1131, -0.6931,  1.1633, -1.2421, -0.2833,  0.7444]],
       grad_fn=<MmBackward0>)


In [60]:
#To standardize the process create 3 matrices wq, wk, wv which will help in the processing of input matrix

class AttnWeights(torch.nn.Module):
    def __init__(self,r,c):
        super().__init__()
        self.wq=torch.nn.Parameter(torch.rand(r,c))
        self.wk=torch.nn.Parameter(torch.rand(r,c))
        self.wv=torch.nn.Parameter(torch.rand(r,c))

    def forward(self,input):
        wq_input=input @ self.wq
        wk_input=input @ self.wk
        wv_input=input @ self.wv

        attn_score= wq_input @ wk_input.H
        attn_weights=torch.softmax((attn_score/(wk_input.shape[-1]**0.5)),dim=-1)
        context_vec=attn_weights @ wv_input

        return context_vec

In [61]:
aw=AttnWeights(8,8)
aw.forward(flattened)

tensor([[-1.0328, -2.9521, -2.4829, -2.2293, -1.8444, -0.7724, -2.0626, -1.0723],
        [ 0.8618, -0.8292, -0.3906,  0.3141, -0.1706,  0.9134, -0.1894, -0.0228],
        [-1.0589, -2.9283, -2.4482, -2.1981, -1.8292, -0.7584, -2.0226, -1.0394],
        [-1.1184, -2.8131, -2.3032, -2.1286, -1.7683, -0.7826, -1.9099, -0.9156]],
       grad_fn=<MmBackward0>)